# Introduction to Python Programming

## Walkthrough: Exercise 4.7 - Plotting with Bokeh

First things first, we need to import the functions that we need from the `bokeh` library. `Bar` to create a bar plot, and `show` to render it. We use `output_notebook` here, to allow the plot to be displayed in this Jupyter Notebook - you might want to use `output_file` instead, which will result in the figure being rendered in an HTML file instead. (See below.)

In [1]:
from bokeh.charts import Bar, output_notebook, show
from bokeh.io import gridplot

Now, we need to read the data from the file into Python. Start by initialising an empty dictionary, which will be populated with all of the data, and an empty list that will be used to store the names of the taxa as we encounter them for the first time.   
Next, read through the file line-by-line. Each time we encounter a line starting with 'Site', we extract the name of the site and initialise another empty dictionary - keyed by that site name - in the dictionary that we created before. This will be used to store the data for this site.   
If the line doesn't start with 'Site', we know that it contains data collected from the site name that we saw most recently, so we need to store it in the dictionary that we created for that site. To do that, first we split the line into the two pieces of information it contains: the taxon id and the recorded count. Then, we store these as a key-value pair in the dictionary for the current site.

In [2]:
inputFile = 'speciesDistribution.txt'
handle = open(inputFile, 'r')

sites = {}
taxa = []

for line in handle.readlines():
    line = line.strip()
    if line.startswith('Site'):
        siteName = line[6:]
        sites[siteName] = {}
    else:
        taxonID, count = line.split('\t')
        if taxonID not in taxa:
            taxa.append(taxonID)
        sites[siteName][taxonID] = int(count)


In the end, we want to output the plots for the sites in alphabetical order, and have the taxon ids sorted alphabetically in each plot too. The names of the sites are the keys of our `sites` dictionary, so we can access them by using the `.keys()` method on that dictionary. Then, this list and the list of taxon ids can be sorted - below we demonstrate two ways in which you can achieve this - the built-in `sorted()` function, and the `.sort()` method of a list object.

In [3]:
siteNames = sites.keys()
siteNames = sorted(siteNames)
taxa.sort()

Now, we need to go through each site and add zeroes into the dictionaries, wherever one of our taxa wasn't observed at a particular site. For each site, we need to loop through each taxon id and check whether it doesn't have a count in the site's dictionary. If it doesn't, we should add an entry for this taxon id, with a count of 0.  
_Note: we could use the `.get()` or `.setdefault()` dictionary methods later, when accessing the values, to use zeroes where we find missing values. You can read more about those, and other dictionary methods, [here](https://docs.python.org/3.5/library/stdtypes.html#mapping-types-dict)._

In [4]:
for site in siteNames:
    for taxonID in taxa:
        if taxonID not in sites[site]:
            sites[site][taxonID] = 0


The `Bar` function, and other plotting functions from `bokeh`, expect an iterable data type to plot from. Here, we will create another dictionary, keyed by site name as before but only containing the counts in a list (ordered by taxon id as before). This is slightly clumsy - given that we have already gone to so much trouble to create our dictionary of dictionaries above - but it is good to have a single structure that holds all of our data, and a separate one that contains only the data for plotting. If you'd like to learn more advanced ways of handling and processing data in Python, I recommend reading about [pandas](http://pandas.pydata.org) and [object class definitions](https://docs.python.org/3.5/tutorial/classes.html#a-first-look-at-classes).

Now we use `output_notebook()` to make sure that our plots can be displayed in this Notebook.

In [5]:
output_notebook()

Loading BokehJS ...

_Note: to save the plot to a file instead, use:_

```Python
output_file('my_amazing_plot.html') # give files descriptive names so you can easily identify them in future
```

Now we can start creating the indivual bar plots, one for each site. We want to display these plots all together, so we should start by storing them in a list. You can read more about how to use the `Bar` function [here](http://bokeh.pydata.org/en/latest/docs/user_guide/charts.html#bar-charts).

In [6]:
plots = []
for site in siteNames:
    data = {}
    data[site] = []
    data['taxa'] = taxa
    for taxon in taxa:
        data[site].append(sites[site][taxon])
    plot = Bar(data, 'taxa', values=site, color='taxa', title=site, xlabel='taxon', ylabel='abundance', tools='')
    plots.append(plot)

The final step is to lay out and display the plots. Here, we will use the `gridplot` function from `bokeh.io`, but there are plenty of other options. Check out the documentation [here](http://bokeh.pydata.org/en/latest/docs/user_guide/layout.html).

In [7]:
layout = gridplot([[plots[0],plots[1]],\
                   [plots[2],plots[3]],\
                   [plots[4],plots[5]],\
                   [plots[6]]])
show(layout)